# Setup

In [1]:
# from google.colab import drive
# drive.mount('/content/drive/')
# %cd drive/MyDrive/skripsi2.8/

# if using hosted runtime uncomment this

!python --version

Python 3.7.9


In [2]:
import tensorflow as tf, matplotlib.pyplot as plt, numpy as np, matplotlib.pyplot as plt, matplotlib.patches as patches, os
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from framework.utils import bbox_utils, data_utils, drawing_utils, eval_utils, io_utils, train_utils
from framework.models import faster_rcnn

# tf.autograph.set_verbosity(0)
# import logging
# logging.getLogger("tensorflow").setLevel(logging.ERROR)

tf.__version__

'2.8.0'

In [3]:
# !pip list

# Read Dataset from TFRecord

In [4]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
IMAGE_SIZE = 512  # make sure had same size with the picture

In [5]:
def read_tfrecord(example):
    tfrecord_format = {
        "filename": tf.io.FixedLenFeature([], tf.string),
        "pic": tf.io.FixedLenFeature([], tf.string),
        "bbox": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    filename = tf.cast(example["filename"], tf.string)
    image = tf.io.parse_tensor(example["pic"], out_type = tf.uint8)
    bbox = tf.io.parse_tensor(example["bbox"], out_type = tf.float32)
    label = tf.io.parse_tensor(example["label"], out_type = tf.int32)
    return {"filename": filename, "image": image, "bbox": bbox, "label": label}

In [6]:
def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames)  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order)  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_tfrecord)
    return dataset

In [7]:
def get_dataset(filenames, train=True):
    dataset = load_dataset(filenames)
    if train:
        dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    return dataset

In [8]:
def read_label_map(label_map_path):
    item_id = None
    item_name = None
    items = {}

    with open(label_map_path, "r") as file:
        for line in file:
            line.replace(" ", "")
            if line == "item{":
                pass
            elif line == "}":
                pass
            elif "id" in line:
                item_id = int(line.split(":", 1)[1].strip())
            elif "name" in line:
                item_name = line.split(":", 1)[1].replace("'", "").replace("\"", "").strip()

            if item_id is not None and item_name is not None:
                items[item_name] = item_id
                item_id = None
                item_name = None

    return items

In [9]:
label_map_path = "./data_preparation/label_map.pbtxt"
label_map_dict = read_label_map(label_map_path)

In [10]:
def get_label_text(result, doc = label_map_dict):
    for key, value in doc.items():
        if(value == result + 1):
            return key
    return "Unpredictable"

In [11]:
image_type = "normal"

train_data = get_dataset(f"./data_preparation/{image_type}_train.tfrecord")

test_data = get_dataset(f"./data_preparation/{image_type}_test.tfrecord", train=False)

train_data

<PrefetchDataset element_spec={'filename': TensorSpec(shape=(), dtype=tf.string, name=None), 'image': TensorSpec(shape=<unknown>, dtype=tf.uint8, name=None), 'bbox': TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), 'label': TensorSpec(shape=<unknown>, dtype=tf.int32, name=None)}>

In [12]:
def show_data(data, n):
    for dat in data.take(n):
        plt.imshow(dat["image"])
        for coord in dat["bbox"]: # bbox is ymin, xmin, ymax, xmax
            coord *= IMAGE_SIZE
            rect = patches.Rectangle(
                (coord[1].numpy(), coord[0].numpy()),  # x1, y1
                coord[3].numpy() - coord[1].numpy(),  # width
                coord[2].numpy() - coord[0].numpy(),  # height
                linewidth = 2, edgecolor = "r", fill = False)
            plt.gca().add_patch(rect)
        plt.show()

In [ ]:
show_data(train_data, 20)

# Init Variable

In [13]:
batch_size = 4
epochs = 1

# backbone = "vgg16"
# backbone = "mobilenet_v2"
backbone = "resnet50"

hyper_params = train_utils.get_hyper_params(backbone)
train_total_item = len(list(train_data))

In [14]:
labels = list(label_map_dict.keys())
print(labels)
# We add 1 class for background
hyper_params["total_labels"] = len(labels) + 1
train_data = train_data.map(lambda data : data_utils.preprocessing_before_frcnn(data, IMAGE_SIZE, IMAGE_SIZE))

['damage']


In [15]:
data_shapes = data_utils.get_data_shapes()
padding_values = data_utils.get_padding_values()
train_data = train_data.padded_batch(batch_size, padded_shapes=data_shapes, padding_values=padding_values)

In [16]:
anchors = bbox_utils.generate_anchors(hyper_params)
frcnn_train_feed = train_utils.faster_rcnn_generator(train_data, anchors, hyper_params)

In [17]:
if (backbone == "vgg16"):
    from framework.models.rpn_vgg16 import get_rpn_model
elif (backbone == "mobilenet_v2"):
    from framework.models.rpn_mobilenet_v2 import get_rpn_model
elif (backbone == "resnet50"):
    from framework.models.rpn_resnet50 import get_rpn_model

# Train Model

In [ ]:
# Load weights
frcnn_weight_path = io_utils.get_weight_path("faster_rcnn", backbone)
frcnn_model_path = io_utils.get_model_path("faster_rcnn", backbone)


load_weights = False
load_model = False
if load_model:
  frcnn_model = tf.keras.models.load_model(frcnn_model_path, custom_objects={
    "RoIBBox": faster_rcnn.RoIBBox,
    "RoIDelta": faster_rcnn.RoIDelta,
    "RoIPooling": faster_rcnn.RoIPooling
  })
  # frcnn_model.compile(optimizer=tf.optimizers.SGD(momentum=7e-1))  # uncomment if you need to recompile
else:
  rpn_model, feature_extractor = get_rpn_model(hyper_params)
  frcnn_model = faster_rcnn.get_model_frcnn(
      feature_extractor, rpn_model, anchors, hyper_params)
  frcnn_model.compile(optimizer=tf.optimizers.SGD(learning_rate=92e-5, momentum=8e-1))
  faster_rcnn.init_model_frcnn(frcnn_model, hyper_params)
  if load_weights:
      frcnn_model.load_weights(frcnn_weight_path)

In [ ]:
log_path = io_utils.get_log_path("faster_rcnn", backbone)
checkpoint_callback = ModelCheckpoint(frcnn_model_path, monitor="loss")
weight_checkpoint_callback = ModelCheckpoint(
    frcnn_weight_path,monitor="loss",
    save_best_only=True, save_weights_only=True)
tensorboard_callback = TensorBoard(log_dir=log_path)

# history_path = io_utils.get_history_path("faster_rcnn", backbone)
# csv_callback = tf.keras.callbacks.CSVLogger(history_path,
#                                             separator=",", append=True)

# total_epoch = 0
## result: : 
# schedule_lr_callback = tf.keras.callbacks.LearningRateScheduler(
#     lambda ep: 1e-5 * 10 ** ((ep + total_epoch) / 30))

In [ ]:
step_size_train = train_utils.get_step_size(train_total_item, batch_size)
history = frcnn_model.fit(frcnn_train_feed,
                steps_per_epoch=step_size_train,
                verbose = 1,
                epochs=epochs,
                callbacks=[weight_checkpoint_callback,
                           checkpoint_callback,
                           tensorboard_callback])

# Learning Rate Hypothesis

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import pandas as pd, numpy as np
rcParams['figure.figsize'] = (18, 8)
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False 

In [ ]:
history_path = io_utils.get_history_path("faster_rcnn", backbone)
hist_data = pd.read_csv(history_path)
history_loss = np.array(hist_data["loss"])

In [ ]:
learning_rates = 1e-5 * (10 ** (np.arange(len(history_loss)) / 30))
plt.semilogx(
    learning_rates, 
    history_loss, 
    lw=3, color='#000'
)
plt.title('Learning rate vs. loss', size=20)
plt.xlabel('Learning rate', size=14)
plt.ylabel('Loss', size=14);

# Evaluate Model

In [18]:
labels = ["bg"] + labels
test_total_item = len(list(test_data))

test_data = test_data.map(lambda data : data_utils.preprocessing_before_frcnn(
                              data, IMAGE_SIZE, IMAGE_SIZE))

test_data = test_data.padded_batch(
    batch_size, padded_shapes=data_shapes, padding_values=padding_values)

In [19]:
load_path = io_utils.get_weight_path("faster_rcnn", backbone)
rpn_model, feature_extractor = get_rpn_model(hyper_params)
frcnn_test_model = faster_rcnn.get_model_frcnn(feature_extractor, rpn_model, anchors, hyper_params, mode="test")
frcnn_test_model.load_weights(load_path)

In [20]:
step_size = train_utils.get_step_size(test_total_item, batch_size)
pred_bboxes, pred_labels, pred_scores = frcnn_test_model.predict(test_data, steps=step_size, verbose=1)

120/120 [==============================] - 18s 112ms/step


In [21]:
stats, hist = eval_utils.evaluate_predictions(test_data, pred_bboxes, pred_labels, pred_scores, labels, batch_size)
print("DONE")

UniqueWithCounts(y=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1])>, idx=<tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 0, 0, 0])>, count=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([4])>)

4
index: 0
-------------------------------
UniqueWithCounts(y=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1])>, idx=<tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 0, 0, 0])>, count=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([4])>)

4
index: 0
-------------------------------
UniqueWithCounts(y=<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 1, -1])>, idx=<tf.Tensor: shape=(16,), dtype=int32, numpy=array([0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0])>, count=<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  6])>)

10
index: 0
-------------------------------
UniqueWithCounts(y=<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 1, -1])>, idx=<tf.Tensor: shape=(16,), dtype=int32, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1])>, count=<tf.Tensor

KeyError: 2

In [ ]:
stats

In [ ]:
hist